<a href="https://colab.research.google.com/github/ahmedmoustafa/AUC-Fundamentals-of-Data-Science-II-Spring-2023/blob/main/examples/maternal-smoking/smoking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maternal Smoking

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_table("https://raw.githubusercontent.com/ahmedmoustafa/AUC-Fundamentals-of-Data-Science-II-Spring-2023/main/examples/maternal-smoking/smoking.tsv")
df.shape

In [ ]:
df.head()

## Q1. Does the mom's smoking pattern affect the newborn birth weight?

### Numerically

Let's first see how many different smoking patterns exist in the dataset. We will use the `groupby` function to group the rows (records) in the `DataFrame` by the `smoke` column (variable). Then count the number of different row per each smoking pattern using the `size` function.

In [ ]:
df.groupby('smoke').size()

From the above, we can see that there are four different smoking patterns, `never` (282 rows), `once_not_now` (60 rows), `until_pregnancy` (52 rows), and `now` (216 rows).

Now, let's look at average baby weights for each of these patterns by grouping them and aggregating them by taking `median` of the `weight`.

In [ ]:
df.groupby('smoke').agg(Median_Baby_Weight = ('weight', 'median')).reset_index()

From the table above, we can see that the moms who smoked during pregnancy (`now`) had babies of the lowest birth weight, average of `114.5`.

### Visually

In [ ]:
sns.boxplot(data = df, x = 'smoke', y = 'weight')

The effects of maternal smoking on newborn weight are clearly illustrated above.

---

## Q2. Does the mom’s race affect the newborn birth weight?


In [ ]:
sns.boxplot(data = df, x = 'mom.race', y = 'weight')

In [ ]:
sns.violinplot(data = df, x = 'mom.race', y = 'weight')

## Q3. Is there a correlation between the mom’s weight and the baby’s weight?


In [ ]:
sns.regplot(data = df, x = 'mom.weight', y = 'weight')

## Q4. Is there a correlation between the dad’s weight and the dad's height?

## Q5. From Q3 and Q4, which is a stronger correlation?

## Q6. Is there a correlation between the mom’s weight and the dad’s weight?

## Q7. On average, does the mom’s weight change across the races?

## Q8. Does mom’s smoking pattern change with the mom’s education?

In [ ]:
counts = df.groupby(['mom.edu', 'smoke']).size().reset_index(name = 'count')
counts

In [ ]:
sns.barplot(data = counts, x = "mom.edu", y = 'count', hue = 'smoke')

In [ ]:
totals = df.groupby(['mom.edu']).size().reset_index(name = 'total')
totals

In [ ]:
merged = counts.merge (totals, on = 'mom.edu')
merged

In [ ]:
normalized = merged.assign(ratio = merged['count'] / merged['total'])
normalized

In [ ]:
sns.barplot(data = normalized, x = "mom.edu", y = 'ratio', hue = 'smoke')

## Q9. Does mom’s smoking pattern change with the family income?

## Q10. Is there a relationship between the mom’s race and the dad’s race?